In [37]:
#Representacion de los estados para la resolaucion del problema de los ascensores. 
#aaa

#Estado Inicial: 
#          * personas indicando la plata actual en la que estan
#ini = ((1, 3, 4), ((0,[]),(0,[])))
ini = (((-1,2), (-1,4), (-1,1),(-1,8),(-1,1)), ((0,0,2,(-1,-1), (1,5,2,(-1,-1)), (1,6,2,(-1,-1)), (2, 10,2,(-1,-1)))))
#Reresentacion des estado final solo de las personas ya que los ascensores no importa doende 
fin = ((0,3), (2,11), (2,12),(0,1), (1,9)) 
####ini = (((0,0), (0,4)),((0,0),(2,12)))
####fin = ((0,1),(0,0))
#                   * Ascensores tambien indicando en la planta actual en la que estan. 


#acaben 


In [38]:
# importamos las cosas que vamos a usar de aima

from search import *

class Ascensores(Problem):
    ''' Clase Ascensores la cual implenta la clase abstracta Problema que nos ayuda a resover el porblema
           con varias funciones de busqueda '''
    def __init__(self,initial, goal):
        '''Metodo que inicializa nustro problema'''
        
        Problem.__init__(self, initial, goal)
        #self.search = initial
        self.goal = goal
        
        
    def actions(self, estado):
        '''Apartir de un estado(n) dado nos lista un conjunto de acciones(a) que le podimos aplicar a n para
        alcanzar otro estado n'  ''' 
        
        listaPersonas = list(estado[0])
        listAscensores = list(estado[1])
        
        acciones = list() 
      
        for i in range (len(listaPersonas)):
            persona = listaPersonas[i] #tupla (ascensor, pisoAct)
            
            for j in range (len(listAscensores)):#pos(   0       1          2     3     3(0)     3(1)  )
                ascensor = listAscensores[j] #tupla (bloque, pisoAct, capacidad, (pasajero1, pasajero2))
                if (ascensor[0] == 0 and ascensor[1] < 4) or (ascensor[0] == 1 and ascensor[1] < 8) or (ascensor[0] == 2 and ascensor[1] < 12):
                    acciones.append((i,j, "Subir ascensor")) #Subir ascensor con restricciones de bloque
                if  (ascensor[0] == 0 and ascensor[1] > 0) or (ascensor[0] == 1 and ascensor[1] > 4) or (ascensor[0] == 2 and ascensor[1] > 8):
                    acciones.append((i,j, "Bajar ascensor")) #Subir ascensor con restricciones de bloque
                if persona[1] == ascensor[1] and persona[1] < 12 and persona[0] == j: #Pos per = Pos as Y persona no en techo Y persona en ascensor
                    acciones.append((i,j, "Bajar pasajero"))           #pordria bajar
                if persona[1] == ascensor[1] and persona[1] > 0 and persona[0] == j: #Pos per = Pos as Y persona no en suelo Y persona en ascensor 
                    acciones.append((i,j, "Subir pasajero"))
                if ascensor[2] > 0 and persona[1] == ascensor[1] and persona[0] != j: #Si la capacidad es mayor que cero y la persona comparte piso con el ascensor
                    acciones.append((i,j, "Montar pasajero"))                         #SE PUEDE CAMBIAR UN PASAJERO DE ASCENSOR
                if 0 < len(ascensor[3]) and persona[1] == ascensor[1] and persona[0] == j: #No se bro, se me ha ocurrido esto, pero amos, que ete tu a saer
                    acciones.append((i,j, "Desmontar pasajero"))
                #en el momento que ascensor este en la misma planta que la persona el arbol se ramifica en dos caminos
                #por lo que se generaran estas dos acciones. Quien decida que accion utilizar será la heuristica 
                #puesto que cogere la que menos heuristica tenga ya que estará mas cerca de la solución
                
        return acciones; 

        
    #accion = (plantaPersona,PlantaAscensor, accion en string)
    def result(self, estado, accion):
        '''Dado un estado(n, nodo actual) y una accion(a) nos permite general un nuevo nodo(n') aplicando la accion dada'''
        listaPersonas = list(estado[0]) #la primera(0)posicion de la tupla de estado la convertimos a una lista
        listAscensores = list(estado[1])# Idema para los ascensores con la segunda posicion (1)

        persona = accion[0] # Extraemos la posicion de la listaen donde se encutra la persona a modificar su posicion en el edificio
        ascensor = accion[1]# Idem con los ascensores 
                #Se hace simplemente para facilitar la lectura del codigo

        actualPersona = list(listaPersonas[persona]) #tupla (bloque, pisoAct, capacidad, (pasajero1, pasajero2))
        actualAscensor = list(listAscensores[ascensor])#idem con el ascensor (ascensor, posAct)

        if accion[2] == "Subir ascensor":
            actualAscensor[1] += 1 # Si es llamar a la posicion actual le asignamos la posicion 
                    #actual de la persona que ha solicitado el ascensor
        elif accion[2] == "Bajar ascensor":
            actualAscensor[1] -= 1 # Si es llamar a la posicion actual le asignamos la posicion 
                    #actual de la persona que ha solicitado el ascensor
        elif accion[2] == "Bajar pasajero": #Si es bajar restamos en uno la posicion actual de ambos(bajan un piso)
            actualPersona[1] -= 1 
            actualAscensor[1] -=1
        elif accion[2] == "Subir pasajero": #Si es Subir sumamos en uno la posicion actual de ambos(sube un piso)
            actualPersona[1] += 1
            actualAscensor[1] += 1
        elif accion[2] == "Montar pasajero":
            actualAscensor[2] -= 1 #La capacidad desciende en uno
            for i in range (len(actualAscensor[3])):
                if(actualAscensor[3][i] == -1):
                    actualAscensor[3][i] = persona
                    break
            #actualAscensor[3].append(persona) #Añado al pasajero a la lista de pasajeros del ascensor
            actualPersona[0] = ascensor
        elif accion[2] == "Desmontar pasajero":
            actualAscensor[2] += 1 #La capacidad desciende en uno
            for i in range (len(actualAscensor[3])):
                if(actualAscensor[3][i] == persona):
                    actualAscensor[3][i] = -1
                    break
            actualAscensor[3].remove(persona) #Añado al pasajero a la lista de pasajeros del ascensor
            actualPersona[0] = -1
        listaPersonas[persona] = tuple(actualPersona)
        listAscensores[ascensor] = tuple(actualAscensor)
        return (tuple(listaPersonas), tuple(listAscensores))

    def goal_test(self, estado): # Devuelve true si se ha alcanzado el estado objetivo 
        cond = True
        for i in range (len(estado[0])):
            cond = cond and (self.goal[i][1] == estado[0][i][1]) #tupla (ascensor, pisoAct,...)
        return cond


In [39]:
estado = ini
accion = (0,0, "Mover ascensor")

l = list(estado)
listaPersonas = list(estado[0]) #la primera(0)posicion de la tupla de estado la convertimos a una lista
listAscensores = list(estado[1])# Idema para los ascensores con la segunda posicion (1)
        
persona = accion[0] # Extraemos la posicion de la listaen donde se encutra la persona a modificar su posicion en el edificio
ascensor = accion[1]# Idem con los ascensores 
        #Se hace simplemente para facilitar la lectura del codigo
        
actualPersona = list(listaPersonas[persona]) #Planta actual donde se encutra la persona (bloque, posAct)
actualAscensor = list(listAscensores[ascensor])#idem con el ascensor (bloque, posAct)
        
if accion[2] == "Mover ascensor":
    actualAscensor[1] = int(actualPersona[1]) # Si es llamar a la posicion actual le asignamos la posicion 
            #actual de la persona que ha solicitado el ascensor
    listaPersonas[persona] = tuple(actualPersona)
    listAscensores[ascensor] = tuple(actualAscensor)
if accion[2] == "Bajar pasajero": #Si es bajar restamos en uno la posicion actual de ambos(bajan un piso)
    actualPersona[1] -= 1 
    actualAscensor[1] -=1
    listaPersonas[persona] = tuple(actualPersona)
    listAscensores[ascensor] = tuple(actualAscensor)
if accion[2] == "Subir pasajero": #Si es Subir sumamos en uno la posicion actual de ambos(sube un piso)
    actualPersona[1] += 1
    actualAscensor[1] +=1
    listaPersonas[persona] = tuple(actualPersona)
    listAscensores[ascensor] = tuple(actualAscensor)
        
print(tuple(listaPersonas), tuple(listAscensores))

TypeError: 'int' object is not iterable

In [40]:
p = Ascensores(ini, fin)
p.initial

(((-1, 2), (-1, 4), (-1, 1), (-1, 8), (-1, 1)),
 (0,
  0,
  2,
  (-1, -1),
  (1, 5, 2, (-1, -1)),
  (1, 6, 2, (-1, -1)),
  (2, 10, 2, (-1, -1))))

In [41]:
p.result((((0, 1), (1, 7), (2, 11)), ((0, 0), (1, 5), (1, 6), (2, 10))), (0, 0, "Mover ascensor"))

(((0, 1), (1, 7), (2, 11)), ((0, 0), (1, 5), (1, 6), (2, 10)))

In [29]:
p.actions(p.initial)

[(0, 0, 'Subir ascensor'),
 (0, 1, 'Subir ascensor'),
 (0, 1, 'Bajar ascensor'),
 (0, 2, 'Subir ascensor'),
 (0, 2, 'Bajar ascensor'),
 (0, 3, 'Subir ascensor'),
 (0, 3, 'Bajar ascensor'),
 (1, 0, 'Subir ascensor'),
 (1, 1, 'Subir ascensor'),
 (1, 1, 'Bajar ascensor'),
 (1, 2, 'Subir ascensor'),
 (1, 2, 'Bajar ascensor'),
 (1, 3, 'Subir ascensor'),
 (1, 3, 'Bajar ascensor'),
 (2, 0, 'Subir ascensor'),
 (2, 1, 'Subir ascensor'),
 (2, 1, 'Bajar ascensor'),
 (2, 2, 'Subir ascensor'),
 (2, 2, 'Bajar ascensor'),
 (2, 3, 'Subir ascensor'),
 (2, 3, 'Bajar ascensor'),
 (3, 0, 'Subir ascensor'),
 (3, 1, 'Subir ascensor'),
 (3, 1, 'Bajar ascensor'),
 (3, 2, 'Subir ascensor'),
 (3, 2, 'Bajar ascensor'),
 (3, 3, 'Subir ascensor'),
 (3, 3, 'Bajar ascensor'),
 (4, 0, 'Subir ascensor'),
 (4, 1, 'Subir ascensor'),
 (4, 1, 'Bajar ascensor'),
 (4, 2, 'Subir ascensor'),
 (4, 2, 'Bajar ascensor'),
 (4, 3, 'Subir ascensor'),
 (4, 3, 'Bajar ascensor')]

In [30]:
p.result(p.initial, (0,0, "Mover ascensor"))

(((-1, 2), (-1, 4), (-1, 1), (-1, 8), (-1, 1)),
 ((0, 0, 2, []), (1, 5, 2, []), (1, 6, 2, []), (2, 10, 2, [])))

In [31]:
# Importamos los modulos pertenecinetes a los algoritmos de busqueda 
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [32]:
#%%time
#breadth_first_graph_search(p).solution()

In [33]:




def linear(node): # dado un estado nos devuelve las personas que todavia no estan en su planta objetivo
# es decir, que estan mal colocadas (como en el puzzle de 8)
    result = 0
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        if goal[i] != estado[i]:
            result += 1
    return result

# Sería como el de distancia hasta su objetivo quitando los if que comprueban las paradas para cambio de ascensor
def paradasTotales(node):
    result = 0
    bloque1 = (0,1,2,3,4)
    bloque2 = (4,5,6,7,8)
    bloque3 = (8,9,10,11,12)
    
    goal = fin
    estado = node.state[0] 
    
    for i in range (len(estado)):
        plantaIni = estado[i][1]
        plantaFin = goal[i][1]
        #podemos ignorar la comporacion de goal[i] == estado[i] ya que el resultado es 0
        result += abs(plantaFin - plantaIni)
        
        if plantaFin != plantaIni: 
        #Hago esto con conjuntos para determinar las paradas que han de hacer los ascensores Lentos
        #Por ejemplo si el inicio esta en el bloque 1 y el objetivo es el bloque3 entonces ha de hacer dos paradas
            if plantaIni < plantaFin: #Sube 
                if plantaFin in bloque2 and plantaIni in bloque1: 
                    result += 1
                elif plantaFin in bloque3 and plantaIni in bloque1:
                    result +=2
                elif plantaFin in bloque3 and plantaIni in bloque2:
                    result += 1
            else: # Baja
                if plantaIni in bloque3 and plantaFin in bloque2: 
                    result += 1
                elif plantaIni in bloque3 and plantaFin in bloque1:
                    result +=2
                elif plantaIni in bloque2 and plantaFin in bloque1:
                    result += 1
                
    return result

def distanciaParaObjetivo(node):
    result = 0
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        #print(i)
        if(goal[i][1] != estado[i][1]):
            result += abs(goal[i][1] - estado[i][1])
    
    return result
        
    
    


In [34]:
#Definimos algunas Hehuristicas 
#euristica de moha: suma de la distancia de los pasajeros montados hasta su planta objetivo y la suma de la distancia de lospasajeros sin montar hasta la ubicacion del ascensor

def heuristicaMoha(node):
    result = 0
    estado = node.state
    goal = fin
    distPersonas = 0
    distElev = 0
    listaPersonas = list(estado[0]) #todas las personas 
    listaAscensores = list(estado[1]) #todos los ascensores

    for i in range (len(listaPersonas)):
        distPersonas += abs(listaPersonas[i][1] - goal[i][1])
        for j in range(len(listAscensores)):
            distElev += abs(listaPersonas[i][1] - listAscensores[j][1])
        result = distPersonas + distElev
    return result

In [35]:
%%time
astar_search(p, heuristicaMoha).solution()

TypeError: unhashable type: 'list'